The data used for training can be downloaded from here:https://XtalPred.godziklab.org/XtalPred/data.tar
The dataset belongs to Adam Godzik Lab,

If you like to use different datasets ensure to correctly change the paths to files.

# Short information about the notebook 
This notebook calculates the basic biochemical features of proteins based on the sequence. The features are saved in csv files.

Moreover, two fasta files will be created with correct and uncorrect fasta sequences, which will be used to separate sequences with nonstards amino acids, as they cannot be handled by NetSurfP and some of the Biopython functions.



# CHANGE THE PATHS

In [1]:
PATH_TO_FASTA = "9_1.fasta"

In [2]:
PATH_TO_IUPRED = "iupred2a/iupred2a.py" # ensure to change it to match your path

# MAIN CODE

In [9]:
import pandas as pd
from Bio import SeqIO
import subprocess
import tempfile
import os
import shutil
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from tqdm import tqdm
#import biolib
#deeptmhmm = biolib.load('DTU/DeepTMHMM')
#nsp3 = biolib.load('DTU/NetSurfP-3') <- there is a possibility to run NetSurfP-3 with biolib, but there is a possibility that the queue will belong

In [10]:
def contains_only_standard_amino_acids(sequence):
    """Check if a sequence contains only standard amino acids, because Biopython will throw an error if it doesn't."""
    standard_amino_acids = set('ACDEFGHIKLMNPQRSTVWY')
    return all(aa in standard_amino_acids for aa in sequence)


def analyze_protein(protein_sequence):
    """Analyze a protein sequence using Biopython."""

    X = ProteinAnalysis(str(protein_sequence))

    return {
        'Length': len(str(protein_sequence)),
        'Gravy': f"{X.gravy():.2f}",
        'Instability_index': f"{X.instability_index():.2f}",
        'Isoelectric_point': f"{X.isoelectric_point():.2f}",
        'C_count': X.count_amino_acids()['C'],
        'M_count': X.count_amino_acids()['M'],
        'W_count': X.count_amino_acids()['W'],
        'Y_count': X.count_amino_acids()['Y'],
        'F_count': X.count_amino_acids()['F'],
        'Molecular_weight': f"{X.molecular_weight():.2f}",
        'Aromaticity': f"{X.aromaticity():.2f}",
        'Helix_fraction': f"{X.secondary_structure_fraction()[0]:.2f}",
        'Turn_fraction': f"{X.secondary_structure_fraction()[1]:.2f}",
        'Sheet_fraction': f"{X.secondary_structure_fraction()[2]:.2f}",
        'Extinction_coeff_reduced': X.molar_extinction_coefficient()[0],
        'Extinction_coeff_oxidized': X.molar_extinction_coefficient()[1],
    }


def predict_transmembrane(protein_sequence):
    """Predicts transmembrane regions with DeepTMHMM. More about here: https://dtu.biolib.com/DeepTMHMM"""
    with tempfile.NamedTemporaryFile(delete=False) as fasta_file:
        fasta_file.write(f">temp\n{protein_sequence}".encode())
        fasta_file_path = fasta_file.name
    
    TMHMM_output = deeptmhmm.cli(args=f'--fasta {fasta_file_path}')
    TMHMM_output.save_files("temp")
    """Example output:
    # Q9NWQ8
    #AA,Beta,Periplasm,Membrane,Inside,Outside,Signal
    0 M,0.0,0.0,0.0,0.0,0.99881,4e-05
    1 G,0.0,0.0,0.0,0.0,0.99881,4e-05
    2 P,0.0,0.0,0.0,0.0,0.99881,4e-05"""
    number_of_TM_fragments = 0
    in_TM = False
    with open("temp/temp_probs.csv") as file:
        lines = file.readlines()
    for line in lines[2:]:
        parts = line.strip().split(",")
        if float(parts[3]) >= 0.5:
            if in_TM == False:
                in_TM = True
        else:
            if in_TM == True:
                in_TM = False
                number_of_TM_fragments += 1
    
    if in_TM == True:
        in_TM = False
        number_of_TM_fragments += 1
    os.remove(fasta_file_path)
    shutil.rmtree("temp")
    return {"TM" : number_of_TM_fragments}
            
        


def get_disorder_regions(protein_sequence):
    """Get disordered regions using IUPred2A. Standard treshold of 0.5 is used."""
    with tempfile.NamedTemporaryFile(delete=False) as fasta_file:
        fasta_file.write(f">temp\n{protein_sequence}".encode())
        fasta_file_path = fasta_file.name

    iupred_output = subprocess.run(
        [PATH_TO_IUPRED, fasta_file_path, "long"],
        capture_output=True,
        text=True
    ).stdout

    os.remove(fasta_file_path)

    disorder_regions = []
    current_region = []
    longest_region = 0
    disorder_total = 0
    for line in iupred_output.splitlines():
        if line.startswith('#'):
            continue
        parts = line.split()
        if len(parts) < 3:
            continue
        residue_index = int(parts[0])
        disorder_score = float(parts[2])
        if disorder_score >= 0.5:
            disorder_total += 1
            if len(current_region) == 0:
                current_region = [residue_index, residue_index]
            else:
                current_region[1]+=1
        else:
            if current_region:
                disorder_regions.append(current_region)
                if current_region[1]-current_region[0]+1 > longest_region:
                    longest_region = current_region[1]-current_region[0]+1
                current_region = []

    if current_region:
        disorder_regions.append(current_region)
        if current_region[1]-current_region[0]+1 > longest_region:
            longest_region = current_region[1]-current_region[0]+1

    return {
        'Disorder_regions': disorder_regions,
        'Longest_disorder_region': longest_region,
        'Total_percentage_disorder': disorder_total/len(protein_sequence)
    }



def process_fasta(fasta_file, calculate_tm = False):
    """Main function to process a fasta file and get all the features. Returns data frame"""
    records = SeqIO.parse(fasta_file, "fasta")
    data = []
    for record in tqdm(records):
        if contains_only_standard_amino_acids(record.seq):
            
            analysis_results = analyze_protein(record.seq)
            if "|" in record.id:
                analysis_results['ID'] = record.id.split("|")[1]
            else:
                analysis_results['ID'] = record.id
            disorder_results = get_disorder_regions(record.seq)
            if calculate_tm is True:
                tms = predict_transmembrane(record.seq)
            else:
                # the original dataset from paper contains only proteins without tranmembrane helices. In this case there is no point in calculating it
                tms = {"TM": 0}
            analysis_results.update(tms)  
            analysis_results.update(disorder_results)
            
            data.append(analysis_results)
            
            with open(f"{fasta_file[:fasta_file.find('.fasta')]}_correct.fasta", "a") as correct:
                correct.write(f">{record.id}\n{record.seq}\n")
                # seqeunces to be used in NetSurfP calculations
            
        else:
            print(f"{record.id} constains nonstard amino acids. It cannot be used")
            with open(f"{fasta_file[:fasta_file.find('.fasta')]}_uncorrect.fasta", "a") as uncorect:
                uncorect.write(f">{record.id}\n{record.seq}\n")
                # keeping an eye on what is throw away from the dataset

    df = pd.DataFrame(data)
    df.set_index('ID', inplace=True) 
    return df




In [11]:
fasta_file = PATH_TO_FASTA 
protein_df = process_fasta(fasta_file)
result_file = f'{fasta_file[:fasta_file.find(".fasta")]}.csv'
protein_df.to_csv(result_file, index=True)
print(f'Results saved to {result_file}')


47it [00:01, 25.76it/s]

Results saved to 9_1.csv


In [12]:
protein_df = pd.read_csv(result_file)
protein_df.set_index('ID', inplace=True)
protein_df

,Length,Gravy,Instability_index,Isoelectric_point,C_count,M_count,W_count,Y_count,F_count,Molecular_weight,Aromaticity,Helix_fraction,Turn_fraction,Sheet_fraction,Extinction_coeff_reduced,Extinction_coeff_oxidized,TM,Disorder_regions,Longest_disorder_region,Total_percentage_disorder
ID,,,,,,,,,,,,,,,,,,,,
A5KZL0,394,-0.55,47.36,8.11,6,4,5,9,18,44760.30,0.08,0.29,0.32,0.35,40910,41285,0,"[[132, 132], [205, 234], [236, 236], [256, 315...",60,0.248731
A0A432XPD4,445,-0.56,36.40,5.51,4,6,8,6,28,50409.79,0.09,0.31,0.31,0.35,52940,53190,0,"[[188, 232], [235, 291], [293, 295], [319, 334...",57,0.307865
A0A7Y4EY46,452,-0.52,43.19,6.33,6,9,6,11,19,51363.73,0.08,0.31,0.30,0.35,49390,49765,0,"[[2, 4], [53, 55], [176, 205], [229, 229], [23...",30,0.181416
A0A7Y3Z4P4,457,-0.51,36.07,8.09,6,6,6,13,19,51338.54,0.08,0.28,0.32,0.35,52370,52745,0,"[[179, 209], [211, 211], [233, 261], [269, 292...",31,0.192560
A0A368NMU0,476,-0.57,40.93,8.42,9,8,6,14,19,54090.75,0.08,0.28,0.31,0.34,53860,54360,0,"[[11, 11], [13, 22], [79, 81], [201, 226], [25...",26,0.199580
A0A240EHC9,477,-0.59,45.17,6.59,9,6,6,14,20,54202.60,0.08,0.30,0.30,0.34,53860,54360,0,"[[131, 132], [201, 231], [233, 233], [256, 284...",31,0.176101
A0A084TCK1,477,-0.61,45.53,6.99,10,7,6,12,19,54073.58,0.08,0.30,0.31,0.32,50880,51505,0,"[[131, 131], [201, 231], [233, 233], [255, 285...",31,0.209644
A0A3G4VIV1,477,-0.59,44.27,8.20,10,7,6,13,19,54137.80,0.08,0.29,0.31,0.33,52370,52995,0,"[[131, 131], [201, 231], [233, 233], [256, 285...",31,0.176101
A5L527,477,-0.59,42.45,7.98,9,7,6,13,19,54075.65,0.08,0.31,0.31,0.33,52370,52870,0,"[[201, 231], [233, 233], [255, 285], [288, 290...",31,0.207547
